<a href="https://colab.research.google.com/github/DonghaeSuh/NLP_Pytorch/blob/main/Model/Seq2Seq_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 간단한 전처리
- 특수 문자 제거
- 특수 토큰 추가 MAX_LENGTH인 25에 맞춰서 시작 토큰과 끝 토큰을 추가 -> 이후 field에서 추가해서 사용
- decoder의 입력에 따른 예측값에 해당하는 dataset을 마련

In [ ]:
import pandas as pd

In [ ]:
ls

drive/  sample_data/


In [ ]:
cd drive/MyDrive/Pytorch\ NLP/data_in/Chatbot

/content/drive/MyDrive/Pytorch NLP/data_in/Chatbot


In [ ]:
ls

ChatbotData.csv  Mecab-ko-for-Google-Colab/


In [66]:
df=pd.read_csv('ChatbotData.csv')
df.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


### 특수 문자 제거

In [67]:
df['Q']=df['Q'].str.replace('[^ a-zA-Z0-9가-힣]',"")
df['A']=df['A'].str.replace('[^ a-zA-Z0-9가-힣]',"")
df.head()

<ipython-input-67-21d281052103>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Q']=df['Q'].str.replace('[^ a-zA-Z0-9가-힣]',"")
<ipython-input-67-21d281052103>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['A']=df['A'].str.replace('[^ a-zA-Z0-9가-힣]',"")


,Q,A,label
0,12시 땡,하루가 또 가네요,0
1,1지망 학교 떨어졌어,위로해 드립니다,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠,0
4,PPL 심하네,눈살이 찌푸려지죠,0


In [ ]:
def decoder_input(sent,Max_len):
  sent=sent.split()
  if len(sent)>Max_len-2: # Max_length=25
    sent=sent[:Max_len-2]
    sent.append('<eos>')
    sent[:0]=['<sos>']
  else:
    sent.append('<eos>')
    sent[:0]=['<sos>']
  
  return " ".join(sent)

In [ ]:
def decoder_output(sent):
  sent=sent.split()
  sent.pop(0)
  return " ".join(sent)

In [68]:
# 알고보니 torchtext의 필드정의에서 <sos>, <eos> token 추가를 지원했다. => 이럴 경우 굳이 미리 추가해놓고 준배해줄 필요가 없다.
df['label']=df['A']
df.head()

,Q,A,label
0,12시 땡,하루가 또 가네요,하루가 또 가네요
1,1지망 학교 떨어졌어,위로해 드립니다,위로해 드립니다
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠,여행은 언제나 좋죠
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠,여행은 언제나 좋죠
4,PPL 심하네,눈살이 찌푸려지죠,눈살이 찌푸려지죠


In [69]:
df.to_csv('2_Fine_ChatbotData.csv',index=False)

In [ ]:
decoder_input('안녕하세요 저는 사람입니다',4)

'<sos> 안녕하세요 저는 <eos>'

In [ ]:
decoder_output('<sos> 안녕하세요 저는 <eos>')

'안녕하세요 저는 <eos>'

In [ ]:
df['A']=df['A'].apply(lambda x : decoder_input(x,25))
df.head()

,Q,A,label
0,12시 땡,<sos> 하루가 또 가네요 <eos>,0
1,1지망 학교 떨어졌어,<sos> 위로해 드립니다 <eos>,0
2,3박4일 놀러가고 싶다,<sos> 여행은 언제나 좋죠 <eos>,0
3,3박4일 정도 놀러가고 싶다,<sos> 여행은 언제나 좋죠 <eos>,0
4,PPL 심하네,<sos> 눈살이 찌푸려지죠 <eos>,0


In [ ]:
df['label']=df['A'].apply(decoder_output)
df.head()

,Q,A,label
0,12시 땡,<sos> 하루가 또 가네요 <eos>,하루가 또 가네요 <eos>
1,1지망 학교 떨어졌어,<sos> 위로해 드립니다 <eos>,위로해 드립니다 <eos>
2,3박4일 놀러가고 싶다,<sos> 여행은 언제나 좋죠 <eos>,여행은 언제나 좋죠 <eos>
3,3박4일 정도 놀러가고 싶다,<sos> 여행은 언제나 좋죠 <eos>,여행은 언제나 좋죠 <eos>
4,PPL 심하네,<sos> 눈살이 찌푸려지죠 <eos>,눈살이 찌푸려지죠 <eos>


In [ ]:
df.to_csv('Fine_ChatbotData.csv',index=False)

# 데이터 불러오기 및 dataset,dataLoader 생성

In [1]:
import pandas as pd

In [2]:
ls

drive/  sample_data/


In [3]:
cd drive/MyDrive/Pytorch\ NLP/data_in/Chatbot

/content/drive/MyDrive/Pytorch NLP/data_in/Chatbot


In [4]:
ls

2_Fine_ChatbotData.csv  Fine_ChatbotData.csv
ChatbotData.csv         Mecab-ko-for-Google-Colab/


In [5]:
train_data=pd.read_csv('2_Fine_ChatbotData.csv')
train_data.head()

,Q,A,label
0,12시 땡,하루가 또 가네요,하루가 또 가네요
1,1지망 학교 떨어졌어,위로해 드립니다,위로해 드립니다
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠,여행은 언제나 좋죠
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠,여행은 언제나 좋죠
4,PPL 심하네,눈살이 찌푸려지죠,눈살이 찌푸려지죠


tokenizer = 공백기준분리 str.split() 사용 예정


In [6]:
## hyperparameter

BATCH_SIZE=2
MAX_LEN=25
EPOCH=50
HIDDEN_DIM=1024
EMB_DIM=256

### 필드 정의 및 Tabulardataset 생성

In [7]:
pip install torchtext==0.6

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
from torchtext import data

In [9]:
Q=data.Field(sequential=True,
             use_vocab=True,
             tokenize=str.split,
             batch_first=True,
             fix_length=MAX_LEN)

A=data.Field(sequential=True,
             use_vocab=True,
             init_token='<sos>',
             eos_token='<eos>',
             tokenize=str.split,
             batch_first=True,
             fix_length=MAX_LEN)
LABEL=data.Field(sequential=True,
                 use_vocab=True,
                 eos_token='<eos>',
                 tokenize=str.split,
                 batch_first=True,
                 fix_length=MAX_LEN)

In [10]:
from torchtext.data import TabularDataset

In [23]:
train_data=TabularDataset('2_Fine_ChatbotData.csv',format='csv',fields=[('question',Q),('answer',A),('label',LABEL)],skip_header=True)
vars(train_data[0])

{'question': ['12시', '땡'],
 'answer': ['하루가', '또', '가네요'],
 'label': ['하루가', '또', '가네요']}

### 사전 정의
- 3개의 Field는 하나의 사전을 공유
- 2개의 special token을 이미 보유한 A를 가지고 만든 다음에 다른 두개에 복사

In [52]:
from collections import defaultdict

In [68]:
Q.build_vocab(train_data,min_freq=5)
A.build_vocab(train_data,min_freq=5)
Q.vocab.stoi.update(A.vocab.stoi)
temp=set(Q.vocab.itos)
Q.vocab.itos=temp.update(A.vocab.itos)

LABEL.vocab=A.vocab
vocab_size=len(A.vocab)

In [ ]:
Q.voacb.itos

In [ ]:
A.vocab.stoi

In [ ]:
LABEL.vocab.stoi

In [ ]:
LABEL.vocab.itos

### 시드 고정 후 데이터 분리

In [71]:
import random
import torch
SEED=123
random.seed(SEED)
torch.manual_seed(SEED)

train_data,val_data=train_data.split(split_ratio=0.9,random_state=random.seed(SEED))

### 데이터 로더 생성

In [14]:
from torchtext.data import Iterator

In [72]:
train_iter=Iterator(train_data,batch_size=BATCH_SIZE)
val_iter=Iterator(val_data,batch_size=BATCH_SIZE)

In [79]:
batch=next(iter(train_iter))
batch.question

tensor([[ 350, 1240,   73,    0,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1],
        [ 136,    0,  301,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1]])

In [80]:
batch.answer

tensor([[  2, 398, 882, 169,   0,   0,   4,   3,   1,   1,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1],
        [  2,  84, 336,   0,   3,   1,   1,   1,   1,   1,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1]])

In [81]:
batch.label

tensor([[398, 882, 169,   0,   0,   4,   3,   1,   1,   1,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1],
        [ 84, 336,   0,   3,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1]])

In [82]:
train_iter=Iterator(train_data,batch_size=BATCH_SIZE)
val_iter=Iterator(val_data,batch_size=BATCH_SIZE)

# 모델 구현